# Final Project Analysis





In [7]:
### Run this cell before continuing. 
library(tidyverse)
library(repr)
options(repr.matrix.max.rows = 6)

In [10]:
game_data <- read_csv ("data/players.csv")
game_data

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
⋮,⋮,⋮,⋮,⋮,⋮,⋮
Amateur,FALSE,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db299bd4fedb06a46ad5bb,0.0,Dylan,Prefer not to say,17
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17
Pro,TRUE,d9473710057f7d42f36570f0be83817a4eea614029ff90cf50d8889cdd729d11,0.2,Ahmed,Other,NA


**Activity #1: Explore Datasets - Preliminary**

Using what you have learnt in weeks 1-4, read the dataset, take a look at it, and write a short description about the dataset.

Some questions you should try to answer:
What is the dataset about?
How many variables are there?
How many observations are there?

data set is about... there are 7 variables, which includes.... there are ... observations...


**Activity #2: Explore Datasets Part 2 - Outcome Variable**

Try to answer these questions now:
Identify the main outcome/categorical/label variable in the dataset.

How many values/groups are in this variable?
How many observations are there in each value/group?

Tip: Think about how you are organising your workbook: add more code and markdown cells (and arranged them!) to keep your notebook neat


**Activity #3: Explore Datasets Part 3 - Visualisations!**

Make some visualisations of the outcome variable:

What does the distribution of the variable look like?
What relationship does it have with some of the other variables?
Tip: Try using a range of box plots, scatterplots, bar charts, line graphs, etc.

**Question: What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?**

In [6]:
# source("cleanup.R")??